In [ ]:
import joblib
import numpy as np 
import mne
import numpy as np
from scipy.stats import skew, kurtosis
from mne.preprocessing import ICA
from mne.time_frequency import psd_array_welch
import pandas as pd
from scipy.fftpack import fft
import pywt  # For wavelet transform
import os

In [ ]:
def preprocess_eeg_data(vhdr_file_path, l_freq=1.0, h_freq=40.0, notch_freq=50):
    """Preprocess EEG data."""
    raw = mne.io.read_raw_brainvision(vhdr_file_path, preload=True)
    
    # Set EOG channels
    eog_channels = ['VPVA', 'VNVB', 'HPHL', 'HNHR']
    raw.set_channel_types({ch: 'eog' for ch in eog_channels if ch in raw.ch_names})
    
    # Apply notch filter and bandpass filter
    raw.notch_filter(freqs=[notch_freq], picks='eeg')
    raw.filter(l_freq=l_freq, h_freq=h_freq, picks='eeg')
    
    # Set EEG reference to average
    raw.set_eeg_reference('average', projection=True)
    
    # ICA for artifact removal
    ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
    ica.fit(raw)
    eog_indices, _ = ica.find_bads_eog(raw)
    ica.exclude = eog_indices
    raw = ica.apply(raw)
    
    # Drop specific channels after ICA
    channels_to_drop = ['Erbs', 'OrbOcc', 'Mass']
    raw.drop_channels([ch for ch in channels_to_drop if ch in raw.ch_names])
    
    return raw


In [ ]:
def slice_and_save_eeg(vhdr_file_path,output_dir,slice_duration=60):
    """Preprocess, slice, and save EEG data into 60-second segments."""
    
    # Extract the base name for saving
    base_name = os.path.splitext(os.path.basename(vhdr_file_path))[0]
    
    # Perform preprocessing
    raw = preprocess_eeg_data(vhdr_file_path)
    
    # Calculate the number of samples per slice
    samples_per_slice = int(slice_duration * raw.info['sfreq'])
    
    # Calculate the number of slices
    num_slices = int(len(raw) / samples_per_slice)
    
    # Slice and save
    for i in range(num_slices):
        start_sample = i * samples_per_slice
        end_sample = (i + 1) * samples_per_slice
        
        # Create a new raw object for the slice
        sliced_raw = raw.copy().crop(tmin=start_sample / raw.info['sfreq'], tmax=end_sample / raw.info['sfreq'], include_tmax=False)
        
        # Construct the filename for the slice
        slice_filename = f"{base_name}_{i + 1}.fif"
        
        # Save the slice in the output directory
        slice_output_path = os.path.join(output_dir, slice_filename)
        sliced_raw.save(slice_output_path, overwrite=True)
        print(f"Slice {i + 1} saved as: {slice_output_path}")

    # Handle the remaining data (if any)
    remaining_samples = len(raw) - (num_slices * samples_per_slice)
    if remaining_samples > 0:
        start_sample = num_slices * samples_per_slice
        remaining_raw = raw.copy().crop(tmin=start_sample / raw.info['sfreq'])
        remaining_filename = f"{base_name}_2.fif"
        # Save the remaining data in the output directory
        remaining_output_path = os.path.join(output_dir, remaining_filename)
        remaining_raw.save(remaining_output_path, overwrite=True)
        print(f"Remaining data saved as: {remaining_output_path}")

In [ ]:
slice_and_save_eeg("dataset_s/mdd/sub-88000489/ses-1/eeg/sub-88000489_ses-1_task-restEC_eeg.vhdr")

In [ ]:
import os

def process_folder(source_folder, destination_folder):
    """
    Processes EO files for all subjects and sessions, saving the features to CSV files.

    Args:
        source_folder (str): Path to the root folder containing subject EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    for sub_id in os.listdir(source_folder):
        subject_path = os.path.join(source_folder, sub_id)
        if not os.path.isdir(subject_path):
            continue
        
        for ses_id in os.listdir(subject_path):
            session_path = os.path.join(subject_path, ses_id, "eeg")
            if not os.path.isdir(session_path):
                continue
            
            for file in os.listdir(session_path):
                if file.endswith("_eeg.vhdr"):
                    inp_path = os.path.join(session_path, file)
                    output_filename = f"{file.replace('.vhdr', '.fif')}"
                    # output_filename = output_filename.replace("task-restcombined", "task-rest_combined")
                    output_path = os.path.join(destination_folder, output_filename)
                    slice_and_save_eeg(inp_path,destination_folder)



In [ ]:
process_folder("dataset_s/mdd","split_fif/mdd")

In [ ]:
def extract_channel_features(raw, fmin=0.5, fmax=50):
    # Select only EEG channels
    raw.pick_types(eeg=True)  # This removes non-EEG channels
    data = raw.get_data()
    channel_names = raw.ch_names
    features = {ch: {} for ch in channel_names}

    # Time-domain features
    for i, ch in enumerate(channel_names):
        features[ch]['mean'] = np.mean(data[i])
        features[ch]['variance'] = np.var(data[i])
        features[ch]['skewness'] = skew(data[i])
        features[ch]['kurtosis'] = kurtosis(data[i])
        features[ch]['peak_to_peak'] = np.ptp(data[i])

        # Fourier Transform (FFT)
        fft_values = np.abs(fft(data[i]))
        features[ch]['fft_mean'] = np.mean(fft_values)
        features[ch]['fft_std'] = np.std(fft_values)
        features[ch]['fft_max'] = np.max(fft_values)
        
        # Wavelet Transform (Morlet)
        wavelet = 'cmor1.5-1.0'
        coeffs, _ = pywt.cwt(data[i], scales=np.arange(1, 129), wavelet=wavelet)
        coeffs = np.abs(coeffs)  # Convert complex values to magnitude
        
        features[ch]['wavelet_energy'] = np.sum(np.square(coeffs))

        # Wavelet Transform (DWT) using Daubechies wavelet (db4) #morle
        # coeffs = pywt.wavedec(data[i], 'db4', level=4)
        # features[ch]['wavelet_energy'] = sum(np.sum(np.square(c)) for c in coeffs)
        
        # features[ch]['wavelet_entropy'] = 0  # Initialize wavelet_entropy
        
        # for c in coeffs:
        #     c = c[np.isfinite(c)]
        #     c_norm = c / (np.sum(np.abs(c)) + 1e-10)
        #     features[ch]['wavelet_entropy'] += -np.sum(c_norm * np.log2(c_norm + 1e-10))

    # Frequency-domain features using PSD
    psd = raw.compute_psd(method='welch', fmin=fmin, fmax=fmax, n_fft=2048)
    psd_data = psd.get_data()
    freqs = psd.freqs
    psd_df = pd.DataFrame(psd_data, columns=freqs, index=channel_names)

    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'slow_alpha': (6, 9), 'alpha': (8, 12),
             'beta': (12, 30), 'gamma': (30, 50)}

    for band, (low, high) in bands.items():
        band_power = psd_df.loc[:, (freqs >= low) & (freqs <= high)].mean(axis=1)
        for ch in channel_names:
            features[ch][f'{band}_power'] = band_power[ch]

    # Frontal Alpha Asymmetry (F3-F4)
    if 'F3' in channel_names and 'F4' in channel_names:
        features['F3_F4_alpha_asymmetry'] = features['F4']['alpha_power'] - features['F3']['alpha_power']

    # Convert features to DataFrame
    features_df = pd.DataFrame(features).T

    return features_df

In [ ]:
def process_and_combine(eo_file_path, ec_file_path, output_file):
    all_features = []

    # Process EO file
    raw_eo = mne.io.read_raw_fif(eo_file_path)
    features_eo = extract_channel_features(raw_eo)
    features_eo['condition'] = 'EO'
    all_features.append(features_eo)

    # Process EC file
    raw_ec = mne.io.read_raw_fif(ec_file_path)
    features_ec = extract_channel_features(raw_ec)
    features_ec['condition'] = 'EC'
    all_features.append(features_ec)

    # Combine EO and EC features
    combined_features = pd.concat(all_features, keys=['EO', 'EC'], names=['condition', 'channel'])
    
    # Save combined features to a single CSV file
    combined_features.to_csv(output_file)
    print(f"Features successfully saved to {output_file}")
    # return combined_features

In [ ]:
process_and_combine("split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif","split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif","split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.csv")

In [ ]:
import os

def process_folder(source_folder, destination_folder):
    """
    Processes EO files for all subjects and sessions, saving the features to CSV files.

    Args:
        source_folder (str): Path to the root folder containing subject EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    output_filename=""
    eo_path=""
    ec_path=""
    print(os.listdir(source_folder))
    for sub_id in os.listdir(source_folder):
        subject_path = os.path.join(source_folder, sub_id)
        if not os.path.isdir(subject_path):
            continue
        
        for ses_id in os.listdir(subject_path):
            session_path = os.path.join(subject_path, ses_id, "eeg")
            if not os.path.isdir(session_path):
                continue
            print(ses_id)
            for file in os.listdir(session_path):
                if file.endswith("EC_eeg_1.fif") or file.endswith("EC_eeg_2.fif"):
                    eo_path = os.path.join(session_path, file)
                elif file.endswith("EO_eeg_1.fif") or file.endswith("EO_eeg_2.fif"):
                    ec_file=file
                    ec_path = os.path.join(session_path, file)
                
                print(eo_path,ec_path)
                output_filename=ec_file.replace("EC_eeg","_eeg_combined.csv")
                process_and_combine(eo_path,ec_path,output_filename)

                
                
                # output_path = os.path.join(destination_folder, output_filename)
                




In [ ]:
import os

def process_folder(source_folder, destination_folder):
    """
    Processes EO and EC files for all subjects and sessions, saving the features to CSV files.

    Args:
        source_folder (str): Path to the root folder containing subject EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    eo_path, ec_path = None, None
    ec_file = None  # Initialize ec_file to avoid the UnboundLocalError

    # Iterate through files in the source folder
    for file in os.listdir(source_folder):
        file_path = os.path.join(source_folder, file)

        if file.endswith("EC_eeg_1.fif") or file.endswith("EC_eeg_2.fif"):
            ec_path = file_path
            ec_file = file  # Store EC file name for output filename generation
        elif file.endswith("EO_eeg_1.fif") or file.endswith("EO_eeg_2.fif"):
            eo_path = file_path

        # Process only when both EO and EC files are found
        if eo_path and ec_path and ec_file:
            output_filename = ec_file.replace("EC_eeg", "eeg_combined.csv")
            output_filepath = os.path.join(destination_folder, output_filename)

            print(f"Processing: \n  EO: {eo_path} \n  EC: {ec_path} \n  Output: {output_filepath}")
            process_and_combine(eo_path, ec_path, output_filepath)

            # Reset paths after processing
            eo_path, ec_path, ec_file = None, None, None


In [ ]:
process_folder("dataset_s/mdd","split_fif/mdd")